In [ ]:
DATASET = 'CIFAR10' # 'CIFAR100' or 'TinyImageNet'
MODEL = 'BT'# 'SimCLR' or 'Dino'

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from utils.data_utils import get_main_loader, get_train_val_loaders
from utils.train_utils import reproducibility, run_training_loop, get_criterion
from utils.model_utils import get_model
from utils.knn_utils import knn_eval

from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import ExponentialLR, CyclicLR

In [ ]:
import torch
import torchvision
import torch.nn as nn

resnet = torchvision.models.resnet18()
backbone = nn.Sequential(*list(resnet.children())[:-1])
model = get_model(MODEL, backbone)

In [ ]:
criterion = get_criterion(MODEL)()
LR = 0.006
MIN_LR = LR
MAX_LR = 0.01
opt = SGD(model.parameters(), lr=LR)
scheduler = CyclicLR(opt, base_lr=MIN_LR, max_lr=MAX_LR)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
N_EPOCHS = 60 # 20 for early-stop models
device

In [ ]:
train_loader, val_loader = get_train_val_loaders(DATASET)

In [ ]:
_, _ = knn_eval(model, train_loader, val_loader, device)

In [ ]:
main_loader = get_main_loader(DATASET, MODEL)

In [ ]:
model = run_training_loop(model, main_loader,criterion, 
                        opt, every_batch=True,
                        epochs=N_EPOCHS, device=device, scheduler=scheduler)

In [ ]:
_, _ = knn_eval(model, train_loader, val_loader, device)